**Import Libraries thats needed from pytorch,torchvision,Keras,Opencv**

In [ ]:
import numpy as np
import pandas as pd 
import os

#pytorch utils
import torch
import torchvision

import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from PIL import Image

%matplotlib inline
from tensorflow.keras.utils import plot_model

# Splitting data
from sklearn.model_selection import train_test_split

# Metrics 
from sklearn.metrics import confusion_matrix, classification_report

# Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
#from tensorflow.keras.applications.resnet import ResNet50

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger


**Get the Base Path for the training Images**

In [ ]:
base_path = '../input/final-trainzip/'
print(os.listdir(base_path))

**Get the Length of the classes as each class is a folder of pictures in Training folder**

In [ ]:
classes = len(os.listdir(os.path.join(base_path,'Train')))
classes

**Get all the folders from the Train folder to a list(train_folders)**

In [ ]:
from glob import glob
train_folders = sorted(glob('../input/final-trainzip/Train/*'))

**Loading, showing and showing random images in a grid using Opencv**

In [ ]:
def load_image(img_path, resize=True):
  img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

  if resize:
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)

  return img

def show_image(img_path):
  img = load_image(img_path)
  plt.imshow(img)
  plt.axis('off')

def show_sign_grid(image_paths):
  images = [load_image(img) for img in image_paths]
  images = torch.as_tensor(images)
  images = images.permute(0, 3, 1, 2)
  grid_img = torchvision.utils.make_grid(images, nrow=11)
  plt.figure(figsize=(24, 12))
  plt.imshow(grid_img.permute(1, 2, 0))
  plt.axis('off');

**Random Images from each folder of the Training Set**

In [ ]:
sample_images = [np.random.choice(glob(f'{tf}/*png')) for tf in train_folders]
show_sign_grid(sample_images)

**Just a Random Image from the Folder 42, 10th Image**

In [ ]:
img_path = glob(f'{train_folders[42]}/*png')[10]
show_image(img_path)

**Getting all the Images from the folder to a data listafter resizing to height and width desired**

In [ ]:
data=[]
labels=[]

height = 30
width = 30
channels = 3
classes = 43
n_inputs = height * width*channels

for i in range(classes) :
    path = "../input/final-trainzip/Train/{0}/".format(i)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print(" ")
            
Cells=np.array(data)
labels=np.array(labels)



In [ ]:
#Printing Shape of the data and the labels
print(f'Shape of the data {Cells.shape}')
print(f'Shape of the labels {labels.shape}')
print(labels)

In [ ]:
# Split into training and validation data 
X_train, X_val, y_train, y_val = train_test_split(Cells, labels, test_size=0.25, random_state=42)

# normalize the input data
X_train = X_train/255.
X_val = X_val/255.

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
#One hot encoding of the data as Ml algorithms cannot work with categorical data.
#All categorical data must be converted to numbers
from keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

**Deep Neural Networks with convolution layers, Max pooling layers, Dense and Drop out layers are picked to customize for this dataset and the output layer activation function is Softmax and all the hidden layers activation function is ReLU.**

**Loss function used is: Categorical Loss function
Optimizer for gradient update used is : Adam**


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(43, activation='softmax'))

#model compilation with loss fn and optimizer of adam
model.compile(
    loss='categorical_crossentropy', 
    optimizer='Adam', 
    metrics=['accuracy']
)


**Training for 10 epochs**

In [ ]:
epochs = 10
history = model.fit(X_train, y_train, batch_size= 64, epochs=epochs,validation_data=(X_val, y_val))

**Plotting Accuracy and Loss**

In [ ]:
import matplotlib.pyplot as plt
print("Training set accuracy is:","{:.2f}%".format(max(history.history['accuracy'])*100))
print("Validation set accuracy is:","{:.2f}%".format(max(history.history['val_accuracy'])*100))


plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

print("Training Loss:","{:.4f}".format(min(history.history['loss'])))
print("Validation Loss:","{:.4f}".format(min(history.history['val_loss'])))
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

**Get the Images from the Test Folder**

In [ ]:
img_folder = '../input/gtsrb-german-traffic-sign/Test'
sample_images = [np.random.choice(glob(f'{img_folder}/*png')) for i in img_folder]
show_sign_grid(sample_images)

**Get the Test.csv file and Iterate through Images**

In [ ]:
y_test=pd.read_csv("../input/gtsrb-german-traffic-sign/Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image=cv2.imread('../input/gtsrb-german-traffic-sign/Test/'+ f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
pred = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Test score accuracy is:","{:.2f}%".format(accuracy_score(y_test, pred)*100))

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
def show_confusion_matrix(confusion_matrix):

  cm = confusion_matrix.copy()

  cell_counts = cm.flatten()

  cm_row_norm = cm / cm.sum(axis=1)[:, np.newaxis]

  row_percentages = ["{0:.2f}".format(value) for value in cm_row_norm.flatten()]

  cell_labels = [f"{cnt}\n{per}" for cnt, per in zip(cell_counts, row_percentages)]
  cell_labels = np.asarray(cell_labels).reshape(cm.shape[0], cm.shape[1])

  df_cm = pd.DataFrame(cm_row_norm)

  hmap = sns.heatmap(df_cm, annot=cell_labels, fmt="", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True Class')
  plt.xlabel('Predicted Class')
  plt.title('Confusion Matrix')


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
cm = confusion_matrix(y_test,pred)
show_confusion_matrix(cm)